# Weather Predictions blender

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [3]:
os.environ['THEANO_FLAGS'] = 'device=cpu'

import numpy as np
import pandas as pd

from data_utils import get_id_type_list_for_class, GENERATED_DATA, OUTPUT_PATH
from test_utils import create_submission

Using Theano backend.


### Load predictions on trainval dataset

In [6]:
from data_utils import unique_tags, get_label

target_tags = ['target_' + t for t in unique_tags]
val_predictions_resnet_filepath = os.path.join(GENERATED_DATA, "val_predictions_ResNet50_2017-07-18-14-50.csv")
val_predictions_squeezenet_filepath = os.path.join(GENERATED_DATA, "val_predictions_SqueezeNet21_2017-07-13-22-23.csv")

val_predictions_vgg19_filepath = os.path.join(GENERATED_DATA, "val_predictions_deep_model_vgg19_train_LB092919_prob.csv")
val_predictions_weirdcnn_filepath = os.path.join(GENERATED_DATA, "val_predictions_deep_model_train_LB092655_prob.csv")

def get_val_predictions_df(val_predictions_filepath, search_prefix="val_predictions_*.csv"):
    if not os.path.exists(val_predictions_filepath):
        val_predictions_csv = glob(os.path.join(OUTPUT_PATH, search_prefix))
        df = pd.read_csv(val_predictions_csv[0]).dropna()
        for filepath in val_predictions_csv[1:]:
            df = pd.concat([df, pd.read_csv(filepath).dropna()])
        df.reset_index(inplace=True)   
        df.drop('index', axis=1, inplace=True)
        df['image_id'] = df['image_name'].apply(lambda x: int(x[len('train_'):]))    
        for t in target_tags:
            df[t] = ''
        def fill_target_tags(row):
            image_id = row[0]
            labels = get_label(image_id, "Train_jpg")
            row[1:] = labels    
            return row
        cols = ['image_id', ] + target_tags
        df[cols] = df[cols].apply(fill_target_tags, axis=1)

        df.to_csv(val_predictions_filepath, index=False)
        val_predictions_df = df
        df = None    
    else:
        val_predictions_df = pd.read_csv(val_predictions_filepath)
    return val_predictions_df


val_predictions_resnet_df = get_val_predictions_df(val_predictions_resnet_filepath, "val_predictions_ResNet50*_2017-07-18-14-50.csv")
val_predictions_squeezenet_df = get_val_predictions_df(val_predictions_squeezenet_filepath, "val_predictions_SqueezeNet21*_2017-07-18-14-50.csv")
val_predictions_vgg19_df = get_val_predictions_df(val_predictions_vgg19_filepath, "vgg19/deep_model_vgg19_train_LB092919_prob.csv")
val_predictions_weirdcnn_df = get_val_predictions_df(val_predictions_weirdcnn_filepath, "custom_weird_model/deep_model_train_LB092655_prob.csv")


In [7]:
print(len(val_predictions_resnet_df))
val_predictions_resnet_df.head()

40448


,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,train_18872,0.762381,0.007814,0.023866,0.002480,0.001474,0.983301,0.000370,0.002931,0.932843,...,0,1,1,0,0,1,0,0,1,1
1,train_10562,0.106635,0.000834,0.002179,0.001490,0.001146,0.000112,0.003351,0.001014,0.020308,...,0,0,0,0,1,1,0,0,0,0
2,train_28763,0.196588,0.004786,0.009777,0.002631,0.001869,0.019417,0.105915,0.004077,0.084854,...,0,1,1,0,1,1,0,0,0,0
3,train_31735,0.084816,0.001532,0.013537,0.010144,0.014883,0.633817,0.002344,0.002039,0.045785,...,0,0,0,0,0,1,0,0,0,0
4,train_320,0.106061,0.001101,0.004905,0.004541,0.001737,0.000195,0.000810,0.001513,0.102018,...,0,0,0,0,1,1,0,0,0,0


In [8]:
print(len(val_predictions_squeezenet_df))
val_predictions_squeezenet_df.head()

40320


,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,train_15438,0.986593,1.800301e-07,0.016957,3.479813e-08,2.728809e-07,0.986830,1.494415e-08,2.339067e-04,0.128825,...,0,1,0,1,0,1,1,0,0,0
1,train_12863,0.024612,7.483472e-08,0.000639,8.198895e-03,1.618286e-03,0.997336,1.701083e-08,9.840907e-09,0.007981,...,0,0,0,0,0,1,0,0,0,0
2,train_10519,0.721716,5.289071e-05,0.014284,1.069636e-04,1.935463e-03,0.000348,2.059214e-05,3.164162e-05,0.509124,...,0,0,0,0,1,1,0,0,0,0
3,train_17178,0.002469,9.790796e-11,0.000020,1.204151e-03,8.132902e-05,0.998747,8.051727e-10,1.947120e-11,0.000470,...,0,0,0,0,0,1,0,0,0,0
4,train_5177,0.066280,3.975656e-07,0.001354,5.240596e-04,2.015755e-04,0.348959,4.480021e-04,5.901127e-07,0.016081,...,0,0,0,0,0,1,0,0,0,0


In [14]:
len(val_predictions_resnet_df['image_name'].unique()), len(val_predictions_squeezenet_df['image_name'].unique()), 

(27158, 31357)

In [10]:
from metrics import score

def get_optimal_thresholds(y_true, y_preds):
    best_thresholds = [0.0]*len(unique_tags)    
    best_score = 0
    thrs = np.arange(0.0, 1.0, 0.01)    
    for i, tag in enumerate(unique_tags):
        print("%s : best_score=" % tag, end="")
        thresholds = list(best_thresholds)
        for thr in thrs:            
            thresholds[i] = thr
            s = score(y_true, y_preds > thresholds)
            if s > best_score:
                best_score = s
                best_thresholds[i] = thr
        print("%f, best_threshold=%f" % (best_score, best_thresholds[i]))
    return best_thresholds, best_score

## Train xgboost trees to make better predict weather

### Train to predict weather classes : 

In [37]:
weather_tags = ['clear', 'cloudy', 'haze', 'partly_cloudy']
weather_target_tags = ['target_%s' % t for t in weather_tags]
weather_labels = np.where(np.isin(unique_tags, weather_tags))

In [38]:
weather_labels, [(i, t) for i, t in enumerate(unique_tags)], weather_target_tags

((array([ 5,  6, 10, 11]),),
 [(0, 'agriculture'),
  (1, 'artisinal_mine'),
  (2, 'bare_ground'),
  (3, 'blooming'),
  (4, 'blow_down'),
  (5, 'clear'),
  (6, 'cloudy'),
  (7, 'conventional_mine'),
  (8, 'cultivation'),
  (9, 'habitation'),
  (10, 'haze'),
  (11, 'partly_cloudy'),
  (12, 'primary'),
  (13, 'road'),
  (14, 'selective_logging'),
  (15, 'slash_burn'),
  (16, 'water')],
 ['target_clear', 'target_cloudy', 'target_haze', 'target_partly_cloudy'])

In [12]:
val_predictions_df = val_predictions_vgg19_df

In [13]:
val_predictions_df.head()

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,0.003752,1.428458e-09,0.000088,0.000230,0.000018,0.581344,2.108085e-05,2.150840e-09,0.001460,0.000500,...,0,0,0,1,0,1,0,0,0,0
1,0.799365,1.561381e-04,0.009740,0.000171,0.000109,0.998879,1.573375e-06,1.418633e-04,0.128822,0.038553,...,0,0,0,0,0,1,0,0,0,1
2,0.001674,1.083242e-09,0.000018,0.000292,0.000128,0.999015,1.269098e-06,8.769292e-11,0.000902,0.000416,...,0,0,0,0,0,1,0,0,0,0
3,0.005073,3.921103e-08,0.000086,0.006642,0.000504,0.998185,2.577778e-06,3.010564e-09,0.002385,0.002778,...,0,0,0,0,0,1,0,0,0,0
4,0.879332,1.331866e-03,0.017324,0.001006,0.000401,0.998223,4.311219e-07,3.768842e-04,0.343372,0.786686,...,0,0,1,0,0,1,1,0,0,0


In [14]:
import xgboost as xgb

#### A simple try of xgb

In [16]:
from sklearn.model_selection import KFold

In [17]:
n_folds = 3

In [103]:
m = val_predictions_df[weather_target_tags].sum(axis=1) == 1

In [106]:
kf = KFold(n_splits=n_folds)

trainval_x = val_predictions_df[m][unique_tags].values
trainval_y = val_predictions_df[m][weather_target_tags].values



d = {
    (1, 0, 0, 0) : 0,
    (0, 1, 0, 0) : 1,    
    (0, 0, 1, 0) : 2,   
    (0, 0, 0, 1) : 3,    
}

def vector_to_index(trainval_y):
    
    output = np.zeros(len(trainval_y), dtype=np.uint8)
    for i, v in enumerate(trainval_y):        
        output[i] = d[tuple(v.tolist())]
    return output

trainval_y_ = vector_to_index(trainval_y)
    
for train_index, test_index in kf.split(trainval_x):
    train_x, val_x = trainval_x[train_index], trainval_x[test_index]
    train_y, val_y = trainval_y_[train_index], trainval_y_[test_index]
    
#     print(train_x.shape, train_y.shape)
#     print(train_x[:5, :], train_y[:5])
#     print(val_x[:5, :], val_y[:5])
    break

In [114]:
params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
    "eval_metric": "error",
    "learning_rate": 0.01,
    "tree_method": 'exact',
    "n_estimators": 150,
    "max_depth": 3,
#     "subsample": subsample,
#     "colsample_bytree": colsample_bytree,
    "silent": False,    
    "num_class": len(weather_target_tags)
}
num_boost_round = 2500
early_stopping_rounds = 100

In [115]:
# sumpw = val_predictions_df[target_tags[tag_index]].sum()
# sumnw = len(val_predictions_df[target_tags[tag_index]]) - sumpw    
# scale_pos_weight = sumnw * 1.0 / sumpw

# params['scale_pos_weight'] = scale_pos_weight

In [116]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x, label=val_y)

In [117]:
watchlist = ((dtrain, 'train'), (dval, 'eval'))

In [118]:
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

XGBoostError: b'[02:32:23] src/metric/elementwise_metric.cc:28: Check failed: preds.size() == info.labels.size() (107940 vs. 26985) label and prediction size not match, hint: use merror or mlogloss for multi-class classification\n\nStack trace returned 2 entries:\n[bt] (0) 0   libxgboost.so                       0x000000011588d269 _ZN4dmlc15LogMessageFatalD1Ev + 41\n[bt] (1) 1   libstdc++.6.dylib                   0x00000001163b3f80 _ZTVNSt7__cxx1115basic_stringbufIcSt11char_traitsIcESaIcEEE + 16\n'

In [25]:
y_preds_ = gbm.predict(xgb.DMatrix(trainval_x), ntree_limit=gbm.best_iteration+1)

In [26]:
search_best_threshold(y_true[:, tag_index], y_preds_)

(0.54000000000000004, 0.25030916502542022)

In [28]:
search_best_threshold(y_true[:, tag_index], val_predictions_df[unique_tags].values[:, tag_index])

(0.080000000000000002, 0.19706884016377763)

In [30]:
thr1 = 0.540
thr2 = 0.08
st = 40
end = 80
print((y_preds_[st:end] > thr1).astype(np.uint8)) 
print((val_predictions_df[unique_tags].values[st:end, tag_index] > thr2).astype(np.uint8))
print(y_true[st:end, tag_index])

[0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0
 0 0 0]
[0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0
 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [315]:
score(y_true[:, tag_index], df[unique_tags[tag_index]].values > 0.09)

0.19568799780280144

Run CV with a random search of optimal parameters

In [68]:
import matplotlib.pylab as plt
%matplotlib inline

In [29]:
def generate_params(iter_num):
    if iter_num > 0:
        for z in range(iter_num):
            print("\n-- Iteration: {}".format(z))
            eta = np.random.uniform(0.05, 0.001)
            max_depth = np.random.randint(2, 6)
            subsample = np.random.uniform(0.5, 0.95)
            colsample_bytree = np.random.uniform(0.5, 0.95)
            yield eta, max_depth, subsample, colsample_bytree
    else:
        eta = 0.05
        max_depth = 3
        subsample = 0.8204967474962096
        colsample_bytree = 0.7089159774987868
        yield eta, max_depth, subsample, colsample_bytree

In [53]:
#eta_values = [0.05, 0.025, 0.01, 0.0075, 0.005, 0.0025, 0.001]

eval_metric = 'logloss'


best_params_for_tag_index = {}
for tag_index in range(0, len(unique_tags)):
    
    print("\n----------------\n Tag index: {}".format(tag_index))    
    seed = 2017
    n_folds = 5
    dtrainval = xgb.DMatrix(trainval_x, label=trainval_y[:, tag_index], feature_names=unique_tags)

    best_params_for_tag_index[tag_index] = {
        'test-%s-mean' % eval_metric: 1e10,
        'params': None,
    }
    
    sumpw = val_predictions_df[target_tags[tag_index]].sum()
    sumnw = len(val_predictions_df[target_tags[tag_index]]) - sumpw    
    scale_pos_weight = sumnw * 1.0 / sumpw
    
    iter_num = 15
#     iter_num = -1   
    gen = generate_params(iter_num)
    z = 0
    for (eta, max_depth, subsample, colsample_bytree) in gen:
        z += 1
        seed += z-1
        print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth,subsample,colsample_bytree))
        params = {
            "objective": "binary:logistic",
            "booster": "gbtree",
            "eval_metric": eval_metric
            ,
            "eta": eta,
            "tree_method": 'exact',
            "max_depth": max_depth,
            "subsample": subsample,
            "colsample_bytree": colsample_bytree,
            "silent": 1,
            "seed": seed, 
            "scale_pos_weight": scale_pos_weight,
        }
        num_boost_round = 2000
        early_stopping_rounds = 100

        cvresult = xgb.cv(params, dtrain=dtrainval,
                           seed=params['seed'], 
                           num_boost_round=num_boost_round, 
                           early_stopping_rounds=early_stopping_rounds, nfold=n_folds, verbose_eval=False)
                
        min_test_logloss_mean = cvresult['test-%s-mean' % params['eval_metric']].min()
        if best_params_for_tag_index[tag_index]['test-%s-mean' % params['eval_metric']] > min_test_logloss_mean:
            best_params_for_tag_index[tag_index]['test-%s-mean' % params['eval_metric']] = min_test_logloss_mean
            best_params_for_tag_index[tag_index]['params'] = params
            print("Best cv result: ", cvresult.loc[cvresult.index[-1], :])
            print("Best params: ", params)
            
        


----------------
 Tag index: 0

-- Iteration: 0
XGBoost params. ETA: 0.023096741976626596, MAX_DEPTH: 2, SUBSAMPLE: 0.9294778786611162, COLSAMPLE_BY_TREE: 0.684612705671509
Best cv result:  test-logloss-mean     0.246641
test-logloss-std      0.009194
train-logloss-mean    0.233773
train-logloss-std     0.002104
Name: 1209, dtype: float64
Best params:  {'seed': 2017, 'scale_pos_weight': 2.2825856875356183, 'booster': 'gbtree', 'subsample': 0.9294778786611162, 'eta': 0.023096741976626596, 'silent': 1, 'objective': 'binary:logistic', 'colsample_bytree': 0.684612705671509, 'tree_method': 'exact', 'eval_metric': 'logloss', 'max_depth': 2}

-- Iteration: 1
XGBoost params. ETA: 0.023096741976626596, MAX_DEPTH: 2, SUBSAMPLE: 0.9294778786611162, COLSAMPLE_BY_TREE: 0.684612705671509

-- Iteration: 2
XGBoost params. ETA: 0.020594509346487103, MAX_DEPTH: 3, SUBSAMPLE: 0.8326168479972074, COLSAMPLE_BY_TREE: 0.8939410323228981
Best cv result:  test-logloss-mean     0.245555
test-logloss-std      0

KeyboardInterrupt: 

In [75]:
import pickle 
now = datetime.now()

xgb_best_params_filepath = os.path.join(GENERATED_DATA, 'xgb_best_params_%s.pkl' % str(now.strftime("%Y-%m-%d-%H-%M")))

with open(xgb_best_params_filepath, 'wb') as handle:
    pickle.dump(best_params_for_tag_index, handle, protocol=pickle.HIGHEST_PROTOCOL)    

In [36]:
best_params_for_tag_index = _best_params_for_tag_index

Train 17 binary classifiers

In [37]:
from sklearn.model_selection import train_test_split

for tag_index in range(len(unique_tags)):
    
    print("\n----------------\n Tag index: {}".format(tag_index))
    train_x, val_x, train_y, val_y = train_test_split(trainval_x, trainval_y, train_size=0.85)
    dtrain = xgb.DMatrix(train_x, train_y[:, tag_index])
    dval = xgb.DMatrix(val_x, val_y[:, tag_index])
    watchlist = [(dtrain, 'train'), (dval, 'eval')]
    params = best_params_for_tag_index[tag_index]['params']
    num_boost_round = 2500    
    early_stopping_rounds = 12

    gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=False)                
    print("Best score: ", gbm.best_score)
    best_params_for_tag_index[tag_index]['gbm'] = gbm



----------------
 Tag index: 0
Best score:  0.108135

----------------
 Tag index: 1
Best score:  0.010417

----------------
 Tag index: 2
Best score:  0.166832

----------------
 Tag index: 3
Best score:  0.08168

----------------
 Tag index: 4
Best score:  0.062996

----------------
 Tag index: 5
Best score:  0.053406

----------------
 Tag index: 6
Best score:  0.022817

----------------
 Tag index: 7
Best score:  0.034226

----------------
 Tag index: 8
Best score:  0.205192

----------------
 Tag index: 9
Best score:  0.095734

----------------
 Tag index: 10
Best score:  0.084821

----------------
 Tag index: 11
Best score:  0.03588

----------------
 Tag index: 12
Best score:  0.058036

----------------
 Tag index: 13
Best score:  0.092097

----------------
 Tag index: 14
Best score:  0.076058

----------------
 Tag index: 15
Best score:  0.168485

----------------
 Tag index: 16
Best score:  0.100198


Compute best thresholds

In [38]:
best_thresholds = {}

for tag_index, tag in enumerate(unique_tags):
        
    dmat = xgb.DMatrix(trainval_x)
    gbm = best_params_for_tag_index[tag_index]['gbm']
    y_preds_ = gbm.predict(dmat, ntree_limit=gbm.best_iteration+1)        
    
    best_thresholds[tag], best_score = search_best_threshold(y_true[:, tag_index], y_preds_)
    print("%s | best threshold : %f with score: %f" % (tag, best_thresholds[tag], best_score))


agriculture | best threshold : 0.490000 with score: 0.893838
artisinal_mine | best threshold : 0.540000 with score: 0.800101
bare_ground | best threshold : 0.650000 with score: 0.481097
blooming | best threshold : 0.580000 with score: 0.348643
blow_down | best threshold : 0.530000 with score: 0.152452
clear | best threshold : 0.220000 with score: 0.979285
cloudy | best threshold : 0.630000 with score: 0.916326
conventional_mine | best threshold : 0.540000 with score: 0.325991
cultivation | best threshold : 0.520000 with score: 0.688206
habitation | best threshold : 0.520000 with score: 0.780279
haze | best threshold : 0.530000 with score: 0.778034
partly_cloudy | best threshold : 0.500000 with score: 0.949168
primary | best threshold : 0.250000 with score: 0.990040
road | best threshold : 0.500000 with score: 0.861935
selective_logging | best threshold : 0.710000 with score: 0.462641
slash_burn | best threshold : 0.540000 with score: 0.221661
water | best threshold : 0.450000 with scor

Boost test predictions

In [39]:
predictions_csv = glob(os.path.join(OUTPUT_PATH, "predictions_*2017-07-13-19-21.csv"))

prediction_df = pd.read_csv(predictions_csv[0]).dropna()
for filepath in predictions_csv[1:]:
    prediction_df = pd.concat([prediction_df, pd.read_csv(filepath).dropna()])
prediction_df.reset_index(inplace=True)   
prediction_df.drop('index', axis=1, inplace=True)

In [40]:
prediction_df.head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,test_40308,0.980499,5.343570e-06,0.012761,1.083201e-06,1.659149e-05,0.026706,6.184839e-06,1.213845e-04,0.247138,0.230338,0.002780,0.955976,0.994307,0.509100,6.428061e-05,5.902686e-03,0.049497
1,test_36168,0.008954,1.964079e-09,0.000126,4.806151e-04,5.747763e-05,0.990458,1.741330e-08,7.339324e-10,0.002185,0.001168,0.006540,0.000672,0.999985,0.002998,2.039154e-04,6.675383e-07,0.004990
2,test_6070,0.998530,1.298099e-10,0.004025,1.012986e-10,1.425290e-09,0.999737,2.468224e-13,2.351865e-06,0.095436,0.819567,0.000283,0.000036,0.947903,0.991666,2.751586e-07,2.056249e-04,0.032477
3,test_5483,0.003073,3.214428e-09,0.000053,3.451585e-02,1.510115e-03,0.997953,1.104697e-08,2.678857e-10,0.001081,0.000312,0.000302,0.001118,0.999984,0.001043,3.596786e-03,6.073845e-07,0.002270
4,test_5532,0.004007,6.949826e-08,0.000144,1.529595e-01,6.396817e-03,0.998306,3.806774e-08,3.966572e-09,0.001861,0.000672,0.000304,0.000979,0.999972,0.002848,2.929822e-02,2.036211e-06,0.003847


In [41]:
y_preds_init = prediction_df[unique_tags].values
y_preds = np.zeros_like(y_preds_init)

for tag_index in range(len(unique_tags)):
    
    print("\n----------------\n Tag index: {}".format(tag_index))
    dtest = xgb.DMatrix(y_preds_init)
    gbm = best_params_for_tag_index[tag_index]['gbm']
    y_preds_ = gbm.predict(dtest, ntree_limit=gbm.best_iteration+1)        
    y_preds[:, tag_index] = y_preds_



----------------
 Tag index: 0

----------------
 Tag index: 1

----------------
 Tag index: 2

----------------
 Tag index: 3

----------------
 Tag index: 4

----------------
 Tag index: 5

----------------
 Tag index: 6

----------------
 Tag index: 7

----------------
 Tag index: 8

----------------
 Tag index: 9

----------------
 Tag index: 10

----------------
 Tag index: 11

----------------
 Tag index: 12

----------------
 Tag index: 13

----------------
 Tag index: 14

----------------
 Tag index: 15

----------------
 Tag index: 16


In [42]:
df = prediction_df.copy()
df[unique_tags] = y_preds

In [43]:
df.head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,test_40308,0.522231,0.45251,0.252149,0.389114,0.455467,0.161270,0.123053,0.455544,0.522468,0.544217,0.443230,0.866382,0.750729,0.503381,0.268825,0.499394,0.248423
1,test_36168,0.477120,0.45251,0.159650,0.389114,0.455467,0.807806,0.122661,0.455544,0.391684,0.410128,0.443108,0.136900,0.729628,0.494106,0.260546,0.451676,0.141758
2,test_6070,0.522231,0.45251,0.222007,0.389114,0.455467,0.830000,0.132560,0.455544,0.527279,0.583079,0.440586,0.190396,0.609088,0.505878,0.259275,0.451676,0.238310
3,test_5483,0.477120,0.45251,0.158208,0.533957,0.478718,0.837238,0.122661,0.455544,0.391450,0.412522,0.435470,0.134498,0.761620,0.494106,0.327320,0.451676,0.127105
4,test_5532,0.477120,0.45251,0.161170,0.601405,0.455475,0.837238,0.122661,0.455544,0.404547,0.414466,0.435470,0.135214,0.761620,0.494106,0.620745,0.451676,0.132539


In [44]:
def compute_mean(df):
    gb = df.groupby('image_name')
    df2 = gb.agg(np.mean).reset_index()
    return df2

In [45]:
mean_df = compute_mean(df)

In [46]:
len(mean_df), len(df)

(61191, 183573)

In [47]:
create_submission(mean_df, info="squeezenet21_blended_3_folds", thresholds=best_thresholds)

## Storages

Last best

In [35]:
_best_params_for_tag_index = {
0: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.9097441644531741,
'eta': 0.012063976013602831,
'eval_metric': 'error',
'max_depth': 3,
'objective': 'binary:logistic',
'scale_pos_weight': 2.2953291651342407,
'seed': 2027,
'silent': 1,
'subsample': 0.8135263419352342,
'tree_method': 'exact'},
'test-error-mean': 0.18432560000000001},
1: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.9247718213959808,
'eta': 0.048774868242772856,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 119.17883755588673,
'seed': 2045,
'silent': 1,
'subsample': 0.8232135887360947,
'tree_method': 'exact'},
'test-error-mean': 0.0096606000000000018},
2: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.8393513697801558,
'eta': 0.04965712898037271,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 45.478386167146972,
'seed': 2083,
'silent': 1,
'subsample': 0.766586509402208,
'tree_method': 'exact'},
'test-error-mean': 0.035527999999999997},
3: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.8393513697801558,
'eta': 0.04965712898037271,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 117.76288659793815,
'seed': 2083,
'silent': 1,
'subsample': 0.766586509402208,
'tree_method': 'exact'},
'test-error-mean': 0.0097225999999999996},
4: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.5079279690320685,
'eta': 0.04557168327562415,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 404.22613065326635,
'seed': 2108,
'silent': 1,
'subsample': 0.8351662492654948,
'tree_method': 'exact'},
'test-error-mean': 0.00248},
5: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.8393513697801558,
'eta': 0.04965712898037271,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 0.42622167984294584,
'seed': 2083,
'silent': 1,
'subsample': 0.766586509402208,
'tree_method': 'exact'},
'test-error-mean': 0.13793419999999998},
6: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.8393513697801558,
'eta': 0.04965712898037271,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 18.122599004031301,
'seed': 2083,
'silent': 1,
'subsample': 0.766586509402208,
'tree_method': 'exact'},
'test-error-mean': 0.032638800000000003},
7: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.5079279690320685,
'eta': 0.04557168327562415,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 383.0,
'seed': 2108,
'silent': 1,
'subsample': 0.8351662492654948,
'tree_method': 'exact'},
'test-error-mean': 0.0027032000000000002},
8: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.8393513697801558,
'eta': 0.04965712898037271,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 8.2033782241497377,
'seed': 2083,
'silent': 1,
'subsample': 0.766586509402208,
'tree_method': 'exact'},
'test-error-mean': 0.22521079999999999},
9: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.8393513697801558,
'eta': 0.04965712898037271,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 10.04960263085777,
'seed': 2083,
'silent': 1,
'subsample': 0.766586509402208,
'tree_method': 'exact'},
'test-error-mean': 0.1460564},
10: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.5079279690320685,
'eta': 0.04557168327562415,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 13.936099277644008,
'seed': 2108,
'silent': 1,
'subsample': 0.8351662492654948,
'tree_method': 'exact'},
'test-error-mean': 0.065190800000000007},
11: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.8393513697801558,
'eta': 0.04965712898037271,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 4.569060773480663,
'seed': 2083,
'silent': 1,
'subsample': 0.766586509402208,
'tree_method': 'exact'},
'test-error-mean': 0.1443574},
12: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.8393513697801558,
'eta': 0.04965712898037271,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 0.080400326906844941,
'seed': 2083,
'silent': 1,
'subsample': 0.766586509402208,
'tree_method': 'exact'},
'test-error-mean': 0.065996799999999994},
13: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.9097441644531741,
'eta': 0.012063976013602831,
'eval_metric': 'error',
'max_depth': 3,
'objective': 'binary:logistic',
'scale_pos_weight': 4.051049170059505,
'seed': 2027,
'silent': 1,
'subsample': 0.8135263419352342,
'tree_method': 'exact'},
'test-error-mean': 0.17645080000000002},
14: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.9247718213959808,
'eta': 0.048774868242772856,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 124.21739130434783,
'seed': 2045,
'silent': 1,
'subsample': 0.8232135887360947,
'tree_method': 'exact'},
'test-error-mean': 0.0087554},
15: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.8393513697801558,
'eta': 0.04965712898037271,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 207.9119170984456,
'seed': 2083,
'silent': 1,
'subsample': 0.766586509402208,
'tree_method': 'exact'},
'test-error-mean': 0.0049351999999999998},
16: {'params': {'booster': 'gbtree',
'colsample_bytree': 0.8393513697801558,
'eta': 0.04965712898037271,
'eval_metric': 'error',
'max_depth': 5,
'objective': 'binary:logistic',
'scale_pos_weight': 4.4593460158418523,
'seed': 2083,
'silent': 1,
'subsample': 0.766586509402208,
'tree_method': 'exact'},
'test-error-mean': 0.25375720000000002}}

In [122]:
_best_params_for_tag_index = {
0: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.38209179999999998},
 1: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.036067200000000001},
 2: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.6334399849809398,
   'eta': 0.15055304521905347,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2019,
   'silent': 1,
   'subsample': 0.6186701247268119,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.086130599999999988},
 3: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5045164695165618,
   'eta': 0.1428101678052025,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.7215305289307261,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.041870600000000001},
 4: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.7089159774987868,
   'eta': 0.20210354378354375,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2017,
   'silent': 1,
   'subsample': 0.8204967474962096,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.015638200000000001},
 5: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.30505919999999997},
 6: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.068178600000000006},
 7: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.7089159774987868,
   'eta': 0.20210354378354375,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2017,
   'silent': 1,
   'subsample': 0.8204967474962096,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.013706000000000001},
 8: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.8579978891053311,
   'eta': 0.18574545610881862,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2044,
   'silent': 1,
   'subsample': 0.8611335020320647,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.29456899999999997},
 9: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5045164695165618,
   'eta': 0.1428101678052025,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.7215305289307261,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.20715659999999997},
 10: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.7089159774987868,
   'eta': 0.20210354378354375,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.8204967474962096,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.1200102},
 11: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.29197859999999998},
 12: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.1244864},
 13: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5045164695165618,
   'eta': 0.1428101678052025,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.7215305289307261,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.3228318},
 14: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.8398837534294123,
   'eta': 0.14522170765829945,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2022,
   'silent': 1,
   'subsample': 0.8734804475952236,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.041907600000000003},
 15: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.586741114680066,
   'eta': 0.08503519422605446,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2052,
   'silent': 1,
   'subsample': 0.9395104239451562,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.027125},
 16: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.38378999999999996}
}

{0: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.38209179999999998},
 1: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.036067200000000001},
 2: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.6334399849809398,
   'eta': 0.15055304521905347,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2019,
   'silent': 1,
   'subsample': 0.6186701247268119,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.0861305999999

Last saved

In [48]:
best_params_for_tag_index = {0: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5829334344789511,
   'eta': 0.024988972967877485,
   'eval_metric': 'logloss',
   'max_depth': 5,
   'objective': 'binary:logistic',
   'scale_pos_weight': 2.2825856875356183,
   'seed': 2038,
   'silent': 1,
   'subsample': 0.7387515922133331,
   'tree_method': 'exact'},
  'test-logloss-mean': 0.24204939999999997},
 1: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5151974580887335,
   'eta': 0.02653115436725616,
   'eval_metric': 'logloss',
   'max_depth': 5,
   'objective': 'binary:logistic',
   'scale_pos_weight': 114.2,
   'seed': 2023,
   'silent': 1,
   'subsample': 0.8162897671482163,
   'tree_method': 'exact'},
  'test-logloss-mean': 0.017903800000000001},
 2: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.9152266627015204,
   'eta': 0.03617273607658269,
   'eval_metric': 'logloss',
   'max_depth': 5,
   'objective': 'binary:logistic',
   'scale_pos_weight': 43.849833147942157,
   'seed': 2122,
   'silent': 1,
   'subsample': 0.6340682509577085,
   'tree_method': 'exact'},
  'test-logloss-mean': 0.095588800000000002},
 3: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5151974580887335,
   'eta': 0.02653115436725616,
   'eval_metric': 'logloss',
   'max_depth': 5,
   'objective': 'binary:logistic',
   'scale_pos_weight': 124.21739130434783,
   'seed': 2023,
   'silent': 1,
   'subsample': 0.8162897671482163,
   'tree_method': 'exact'},
  'test-logloss-mean': 0.042751600000000001}
}